In [1]:
# webpage extraction with beautifulsoup
import urllib2
from bs4 import BeautifulSoup

#specify the url
manual = 'https://haynes.com/en-us/car-manuals/car-makes'

#Query the website and return the html to the variable 'page'
page = urllib2.urlopen(manual)

In [2]:
soup = BeautifulSoup(page, "lxml")

In [ ]:
print soup.prettify()

In [4]:
makes = soup.find_all("div", class_="item-list")[1]
links_makes = makes.find_all("div", class_ = "views-field views-field-nothing")

In [5]:
list_makes = []
prefix = "https://haynes.com"
for make in links_makes:
    list_makes.append(prefix + make.a["href"])    

In [6]:
#Query the website and return the html to the variable 'page'
list_models = []

for link in list_makes:
    page_temp = urllib2.urlopen(link)
    soup_temp = BeautifulSoup(page_temp, "lxml")
    model_temp = soup_temp.find_all("div", class_="item-list")[1]
    links_model = model_temp.find_all("li")
    for model in links_model:
        list_models.append(model.a["href"])


In [10]:
def extract_info(link, result):
    import re
    page_1 = urllib2.urlopen(link)
    soup_1 = BeautifulSoup(page_1)
    title = soup_1.find("h1", class_="product-title").get_text()
    price = soup_1.find("span", class_="price-amount commerce-price-savings-formatter-price").get_text()
            
    body = soup_1.find_all("div", class_="field-item even")
    
    Format = body[2].get_text()
    decription = body[4].get_text() + body[5].get_text()+ body[6].get_text() + body[7].get_text()
    description = decription.encode('utf-8')
    
    table = body[8].get_text()
    part_no = soup_1.findAll('div', text = re.compile('Part number'), attrs = {'class' : 'field-item even'})
    dimensions = soup_1.findAll('div', text = re.compile('Dimensions'), attrs = {'class' : 'field-item even'})
    pages = soup_1.findAll('div', text = re.compile('# of pages'), attrs = {'class' : 'field-item even'})
    ISBN13 = soup_1.findAll('div', text = re.compile('ISBN-13'), attrs = {'class' : 'field-item even'})
    ISBN10 = soup_1.findAll('div', text = re.compile('ISBN-10'), attrs = {'class' : 'field-item even'})
    UPC = soup_1.findAll('div', text = re.compile('UPC'), attrs = {'class' : 'field-item even'})
    publish_date = soup_1.findAll('div', text = re.compile('Publication date'), attrs = {'class' : 'field-item even'})
    language = soup_1.findAll('div', text = re.compile('Language'), attrs = {'class' : 'field-item even'})
    
    reviews = soup_1.find("iframe", class_ = "product-reviews-amazon")
    if reviews is not None:
        reviews = reviews["src"]
    
    result.append([title, price, Format,description, table, part_no, dimensions, pages,ISBN13, ISBN10, UPC, publish_date, language, reviews])

In [11]:
import time
start = time.time()

result = []
for link in list_models:
    # 3 types of links
    # 1-result of a search, go to next level
    if "search?query" in link:
        page_temp = urllib2.urlopen(link)
        soup_temp = BeautifulSoup(page_temp, "lxml")
        model_temp = soup_temp.find_all("div", class_="product-teaser-title")[1]
        for item in model_temp:
            if item is not None:
                #print item
                link_temp = prefix + item["href"]
                extract_info(link_temp, result)
         
    # 2-need to go to next level by "more info"
    elif link[-1].isdigit():
        page_temp = urllib2.urlopen(link)
        soup_temp = BeautifulSoup(page_temp, "lxml")
        link_temp = soup_temp.find_all("div", class_="views-field views-field-view-node")
        for item in link_temp:
            link_temp = prefix + item.a["href"]
            extract_info(link_temp, result)
    
    # 3-final page
    else:
        extract_info(link, result)
        
end = time.time()
print(end - start)
'''
    page_temp = urllib2.urlopen(link)
    soup_temp = BeautifulSoup(page_temp)
    
    model_temp = soup_temp.find_all("div", class_="item-list")[1]
    links_model = model_temp.find_all("li")
    for model in links_model:
        list_models.append(model.a["href"])
'''

<a href="/en-us/chevy-s-10-gmc-sonoma-pick-ups-94-04-haynes-digital-manual-includes-95-04-s-10-blazer-gmc-jimmy-gmc">Chevy S-10 &amp; GMC Sonoma Pick-ups (94-04) Haynes Digital Manual. Includes (95-04) S-10 Blazer &amp; GMC Jimmy, GMC Envoy (98-01) and (96-01) Oldsmobile Bravada/ Isuzu Hombre</a>
<a href="/en-us/chevroletgmc-camionetas-haynes-manual-de-reparaci-n-88-al-98-incluye-suburban-92-al-98-blazer-jimmy">Chevrolet/GMC Camionetas Haynes Manual de Reparación: 88 al 98 incluye Suburban 92 al 98, Blazer &amp; Jimmy (los modelos de tamaño Grande 92 al 94, &amp; Tahoe y Yukon 95 al 98. Todos los motores de gasolina, de 2 y 4 tracciones. (edición española)</a>
<a href="/en-us/chevrolet-gmc-full-size-gas-pick-ups-88-98-haynes-repair-manual">Chevrolet &amp; GMC Full-size Gas Pick-ups (88-98) Haynes Repair Manual</a>
<a href="/en-us/chevrolet-silverado-gmc-sierra-gas-pick-ups-99-06-haynes-repair-manual">Chevrolet Silverado &amp; GMC Sierra Gas Pick-ups (99-06) Haynes Repair Manual</a>
<a 

'\n    page_temp = urllib2.urlopen(link)\n    soup_temp = BeautifulSoup(page_temp)\n    \n    model_temp = soup_temp.find_all("div", class_="item-list")[1]\n    links_model = model_temp.find_all("li")\n    for model in links_model:\n        list_models.append(model.a["href"])\n'

In [ ]:
'''
for item in model_temp:
    print item
    if item:
        print item["href"]
'''

In [18]:
print len(result)

1691


In [13]:
import json

with open("Manuals.json", "w") as writeJSON:
    json.dump(result, writeJSON)

In [24]:
import csv
import pandas as pd

df = pd.DataFrame(result)
df
#df.to_csv('my_csv.csv', index = False, header = False)

'''
with open("Manuals.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(result)
'''
'''
with open("Manuals.csv", "w") as toWrite:
    writer = csv.writer(toWrite, delimiter=",")
    writer.writerow(["title","price", "Format","decription", "table", "part_no", "dimensions", "pages","ISBN13", "ISBN10", "UPC", "publish_date", "language", "reviews"])
    for a in result:
        writer.writerow(a)
'''

'\nwith open("Manuals.csv", "w") as toWrite:\n    writer = csv.writer(toWrite, delimiter=",")\n    writer.writerow(["title","price", "Format","decription", "table", "part_no", "dimensions", "pages","ISBN13", "ISBN10", "UPC", "publish_date", "language", "reviews"])\n    for a in result:\n        writer.writerow(a)\n'

In [ ]:
print soup_temp.prettify()